In [2]:
import os 
import sys 
sys.path.append('../lznestpy')

import nestpy
from random import choices
from nestUtils import *

import matplotlib.pyplot as plt
import numpy as np

from LXeSimulation import *



In [7]:
def simulate_LXe_NEST(EventRateFile, nEvents, g1, g2):
    trueEventRate = np.loadtxt(EventRateFile, skiprows=0,delimiter=',')
    if trueEventRate.shape[0] >= 1500:
        trueEventRate = trueEventRate[:1000,:]
    reco_energies, S1s, spikes, S2s, weights = simulate_LXe(EventRateFile, nEvents, g1, g2)
    cutMask = (S1s >= 3) & (S1s <= 80) & (spikes >= 3)
    return reco_energies[cutMask], weights[cutMask], S1s[cutMask], S2s[cutMask]


def significance(signalRate, bkgRate, exposure):
    return signalRate * exposure / np.sqrt(bkgRate*exposure)


def findXenonSingalRate(eventRateFile, g1, g2):
    eventRate = np.loadtxt(eventRateFile, skiprows=0,delimiter=',')
    energy, weight, S1, S2 = simulate_LXe_NEST(eventRateFile, 1000000, g1, g2)
    hist, edges = np.histogram(energy, bins=np.arange(0, 100), weights=4*0.9*weight)
    total_rate = np.sum(hist)
    return total_rate, hist, edges


def findXenonModelSignificance(eventRateFile, g1=0.114, g2=47.1, exposure=15.3):
    total_rate, hist, edges = findXenonSingalRate(eventRateFile, g1, g2)
    return significance(total_rate, 11*0.05, exposure)










In [9]:
print(findXenonModelSignificance("/Users/yxu/workbenches/XeArLoopworkbench/EventRatesMMA/Xenon/Xe50zTable.csv", 0.114, 47.1, 0.9 ))

4.919326480309632
